# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-df6ce197ae-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Literary Movements and physicists explorative search") 

Consider the following exploratory information need:

> investigate the literary movements and the poets. In particular, compare some physicists in terms of discovers, Nobel Prizes and education.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q43514`| theory of relativity | node |





Also consider

```
wd:Q43514 ?p ?obj .
```

is the BGP to retrieve all **properties of the theory of relativity**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:

1. Who invented the theory of relativity? (return the IRI and the label of the inventor).

2. Identify the BGP for physicist

3. Identify the BGP for Nobel Prize

4. Consider physicists born in Europe. For each european country return the number of physicists who won the Nobel Prize in Physics (the result set must be a list of triples country IRI, label and #physicists).

5. Return the countries which won at least once all the type of Nobel Prizes with the people born there. (the result set must be a list of couples country IRI and label). 

6. Amongst the physicists who won the Nobel Prize in Physics, find who had discovered or invented at least 3 things (the result set must be a list of triples physicist IRI, label and #discoveries ).

7. Amongst the physicists who won the Nobel Prize in Physics, find who taught in the university of his/her city of birth (the result set must be a list of elements of the form physicist IRI, label, university IRI and label).

## Task 1
Who invented the theory of relativity? (return the IRI and the label of the inventor).

In [23]:
# Query 1

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q43514 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('

In [24]:
# Query 2

# From previous query:
# P61 => "discoverer or inventor"
# wd:Q43514 => "theory of relativity"


queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q43514 wdt:P61 ?obj .
   
   # get the label
   ?obj sc:name ?objName .
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q937'), ('objName', 'Albert Einstein')]
1


Final query for this task

In [41]:
# write your final query

# Query 3

# wdt:P61 => "discoverer or inventor"
# wd:Q43514 => "theory of relativity"
# wd:Q937 => 'Albert Einstein'

queryString = """
SELECT DISTINCT ?inventor ?inventorName
WHERE {
   # bind something
   wd:Q43514 wdt:P61 ?inventor .
   
   # get the label
   ?inventor sc:name ?inventorName .
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('inventor', 'http://www.wikidata.org/entity/Q937'), ('inventorName', 'Albert Einstein')]
1


## Task 2
Identify the BGP for physicist

In [26]:
# write your queries

# Query 4

# Finding anything similar to "physicist" related to Einstein.
# wd:Q937 => 'Albert Einstein'

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q937 ?p ?o .
   
   # get the label
   ?p sc:name ?pName .
   ?o sc:name ?oName .
   
   FILTER (REGEX (?pName,"physicist") || REGEX (?oName,"physicist"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q19350898'), ('oName', 'theoretical physicist')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q169470'), ('oName', 'physicist')]
2


In [36]:

# Query 4-improved

# In Query 4 I searched (FILTER) both prediction and object for 'physicist'.
# But since it is supposed that the triple be somthing like "Einstein is physicist", searching prediction for 'physicist' is an unnecessary computation load.
# Thus it is better to search just object. I improved the Query 4 as this one.
# In addition, by changing 'physicist' to '[Pp]ysic]', I found 'Nobel Prize in Physics' that is needed for next task.

# Finding anything similar to "physicist" related to Einstein.
# wd:Q937 => 'Albert Einstein'

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q937 ?p ?o .
   
   # get the label
   ?p sc:name ?pName .
   ?o sc:name ?oName .
   
   FILTER (REGEX (?oName,"[Pp]hysic"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q18362'), ('oName', 'theoretical physics')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q19350898'), ('oName', 'theoretical physicist')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q169470'), ('oName', 'physicist')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q63430342'), ('oName', '1. Solvay Conference on Physics')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q63430346'), ('oName', '5. Solvay Conference on Physics')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q38104'), ('oName', 'Nobel Prize in Phy

In [56]:
# Query 5

# From query 4 I found that:
# wd:Q169470 => 'physicist'
# Query 4 is not efficient. I found that Einestein's occupation is physicist, and I knew wdt:P106 as 'occupation' from given tabel. So a better way was ckecking occupation of Einestein.
# wdt:P106 => 'occupation'
# wd:Q937 => 'Albert Einstein'


queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q937 wdt:P106 ?o .
   
   # get the label
   ?o sc:name ?oName .
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q19350898'), ('oName', 'theoretical physicist')]
[('o', 'http://www.wikidata.org/entity/Q16389557'), ('oName', 'philosopher of science')]
[('o', 'http://www.wikidata.org/entity/Q121594'), ('oName', 'professor')]
[('o', 'http://www.wikidata.org/entity/Q1231865'), ('oName', 'pedagogue')]
[('o', 'http://www.wikidata.org/entity/Q15980158'), ('oName', 'non-fiction writer')]
[('o', 'http://www.wikidata.org/entity/Q1622272'), ('oName', 'university teacher')]
[('o', 'http://www.wikidata.org/entity/Q169470'), ('oName', 'physicist')]
[('o', 'http://www.wikidata.org/entity/Q170790'), ('oName', 'mathematician')]
[('o', 'http://www.wikidata.org/entity/Q205375'), ('oName', 'inventor')]
[('o', 'http://www.wikidata.org/entity/Q36180'), ('oName', 'writer')]
[('o', 'http://www.wikidata.org/entity/Q37226'), ('oName', 'teacher')]
[('o', 'http://www.wikidata.org/entity/Q4964182'), ('oName', 'philosopher')]
[('o', 'http://www.wikidata.org/entity/Q901'), ('oName

Final query for this task

In [6]:
# write your final query

# Query 6

# Note: Binding part of query (wd:Q937 wdt:P106 wd:Q169470 .) is not necessary.
# By Query 5 it is founded that:
# wd:Q169470 => "physicist"

queryString = """
SELECT DISTINCT (wd:Q169470 as ?IRI) ?oName
WHERE {
   # bind something
   wd:Q937 wdt:P106 wd:Q169470 .
   
   # get the label
   wd:Q169470 sc:name ?oName .
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q169470'), ('oName', 'physicist')]
1


## Task 3
Identify the BGP for Nobel Prize

In [32]:
# write your queries

# Query 7

# Finding anything similar to "Nobel" related to Einstein.
# wd:Q937 => 'Albert Einstein'

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q937 ?p ?o .
   
   # get the label
   ?p sc:name ?pName .
   ?o sc:name ?oName .
   
   FILTER (REGEX (?pName,"[Nn]obel") || REGEX (?oName,"[Nn]obel"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q38104'), ('oName', 'Nobel Prize in Physics')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('o', 'http://www.wikidata.org/entity/Q38104'), ('oName', 'Nobel Prize in Physics')]
2


In [37]:
# Query 7-improved

# In Query 7 I searched (FILTER) both prediction and object for '[Nn]obel'.
# But since it is supposed that the triple be somthing like "Einstein awarded Nobel", searching prediction for '[Nn]obel' is an unnecessary computation load.
# Thus it is better to search just object. I improved the Query 7 as this one.

# **** Note that by improving Query 4 (Query 4-improved) I found 'Nobel Prize in Physics', thus Query 7 and Query 7-improved is not necessary.

# Finding anything similar to "Nobel" related to Einstein.
# wd:Q937 => 'Albert Einstein'

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q937 ?p ?o .
   
   # get the label
   ?o sc:name ?oName .
   
   FILTER (REGEX (?oName,"[Nn]obel"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q38104'), ('oName', 'Nobel Prize in Physics')]
1


In [39]:
# Query 8

# By query 7 I found that:
# wd:Q38104 => 'Nobel Prize in Physics'
# It should be an instance or subclass of Nobel Prize. Let check it.

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q38104 wdt:P31 | wdt:P279 ?o .
   
   # get the label
   ?o sc:name ?oName .  
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q7191'), ('oName', 'Nobel Prize')]
[('o', 'http://www.wikidata.org/entity/Q107494520'), ('oName', 'physics award')]
2


Final query for this task

In [40]:
# write your final query

# Query 9

# By Query 8 it is founded that:
# wd:Q7191 => "Nobel Prize"

queryString = """
SELECT DISTINCT (wd:Q7191 as ?IRI) ?oName
WHERE {
   # bind something
   wd:Q38104 ?p wd:Q7191 .
   
   # get the label
   wd:Q7191 sc:name ?oName .
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q7191'), ('oName', 'Nobel Prize')]
1


## Task 4
Consider physicists born in Europe. For each european country return the number of physicists who won the Nobel Prize in Physics (the result set must be a list of triples country IRI, label and #physicists).

In [32]:
# write your queries

# Query 10

# wdt:P27 => 'nationality'
# wd:Q169470 => 'physicist'
# wd:Q38104 => 'Nobel Prize in Physics'
# wdt:P166 => 'award received'  (from Query 7)
# wd:Q937 => 'Albert Einstein'

# Let check if by 'nationality' I can find information needed for filtering European countries.

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q937 wdt:P27 ?o .
   
   # get the label
   ?o sc:name ?oName .
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('o', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('o', 'http://www.wikidata.org/entity/Q39'), ('oName', 'Switzerland')]
[('o', 'http://www.wikidata.org/entity/Q41304'), ('oName', 'Weimar Republic')]
[('o', 'http://www.wikidata.org/entity/Q43287'), ('oName', 'German Empire')]
[('o', 'http://www.wikidata.org/entity/Q7318'), ('oName', 'Nazi Germany')]
[('o', 'http://www.wikidata.org/entity/Q533534'), ('oName', 'Cisleithania')]
6


In [17]:
# Query 11

# I am looking for countries. So wd:Q39 ('Switzerland') could be an answer. Lets get sure.
# wdt:P31 => instance of
# wd:Q39 => 'Switzerland'

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q39 wdt:P31 ?o .
   
   # get the label
   ?o sc:name ?oName .
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3624078'), ('oName', 'sovereign state')]
[('o', 'http://www.wikidata.org/entity/Q6256'), ('oName', 'country')]
2


In [57]:
# Query 12

# Now I am trying to finnd a way for filtering European countries
# wd:Q6256 => 'country'


queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q6256 ?p ?o .
   
   # get the label
   ?o sc:name ?oName .
   ?p sc:name ?pName .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal"), ('o', 'http://www.wikidata.org/entity/Q6988967'), ('oName', 'Portal:Countries')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q5621162'), ('oName', 'Template:Infobox country')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('pName', 'contains administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q10864048'), ('oName', 'first-level administrative country subdivision')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P17'), ('oName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P495'), ('oName', 'country of origin')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('o', 'http://w

In [14]:
# Query 13

# Let check if by 'continent' I can filter European countries.
# wd:Q6256 => 'country'
# wdt:P30 => 'continent'
# wd:Q39 => 'Switzerland'

queryString = """
SELECT DISTINCT  ?o ?oName
WHERE {
   # bind something
   wd:Q39 wdt:P30 ?o .
   
   # get the label
   ?o sc:name ?oName .
   #?p sc:name ?pName .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q46'), ('oName', 'Europe')]
1


In [20]:
# Query 14

# It seems that now I have all information needed. 
# Unfortunately not! Task assignment needs physicist 'born' in Europe. Maybe it was better to find country from where people born. However it is done.
# Now I need info for 'born', for example 'birthplace' or something similar.

# wd:Q6256 => 'country'
# wdt:P30 => 'continent'
# wd:Q39 => 'Switzerland'
# wdt:P27 => 'nationality'
# wd:Q169470 => 'physicist'
# wd:Q38104 => 'Nobel Prize in Physics'
# wdt:P166 => 'award received'
# wd:Q46 => 'Europe'
# wd:Q937 => 'Albert Einstein'

queryString = """
SELECT DISTINCT  ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q937 ?p ?o .

   # get the label
   ?o sc:name ?oName .
   ?p sc:name ?pName .
   
   FILTER(REGEX(?pName,"[Bb]irth"))
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pName', 'place of birth'), ('o', 'http://www.wikidata.org/entity/Q3012'), ('oName', 'Ulm')]
1


In [43]:
# Query 15

# It seems that now I have all information needed. 
# wdt:P19 => 'place of birth'


queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q937 wdt:P19 ?o .

   # get the label
   ?o sc:name ?oName .
   #?p sc:name ?pName .   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3012'), ('oName', 'Ulm')]
1


In [22]:
# Query 16

# It shows that wdt:P19 returns city. I should find a path from it to country. 
# wdt:P19 => 'place of birth'
# wd:Q3012 => Ulm

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q3012 ?p ?o .

   # get the label
   ?o sc:name ?oName .
   ?p sc:name ?pName .   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q8170'), ('oName', 'Tübingen Government Region')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19180675'), ('oName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q4173137'), ('oName', 'Jewish Encyclopedia of Brockhaus and Efron')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q602358'), ('oName', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q97879676'), ('oName', 'Regesta Imperii XIII')]
[('p', 'http://www.wikidat

In [59]:
# Query 17

# Task 4:
# Consider physicists born in Europe.
# For each european country return the number of physicists who won the Nobel Prize in Physics (the result set must be a list of triples country IRI, label and #physicists).

# It is supposed that 'physicist' means that the 'occupation' of the winner should be 'physicist'.
# I do not check if there is any person who won the Nobel Prize in Physics whos occupation is not physicist.
# However, if there is any, they should be excluded from the result. Because task is: "Consider physicists..."

# wdt:P166 => 'award received'
# wd:Q38104 => 'Nobel Prize in Physics'
# wd:P106 => 'occupation'
# wd:Q169470 => 'physicist'
# wdt:P19 => 'place of birth'
# wdt:P17 => 'country'
# wdt:P30 => 'continent'
# wd:Q46 => 'Europe'


queryString = """
SELECT DISTINCT (?c as ?countryIRI) (?cName as ?country) (count(?s) as ?noOfPhysicists)
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
      wdt:P106 wd:Q169470 ;
      wdt:P19 ?pBirth .
  ?pBirth wdt:P17 ?c .
  ?c wdt:P30 wd:Q46 .

   # get the label
   ?c sc:name ?cName .

}
GROUP BY (?c) (?cName)
ORDER BY (?cName)
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('countryIRI', 'http://www.wikidata.org/entity/Q40'), ('country', 'Austria'), ('noOfPhysicists', '2')]
[('countryIRI', 'http://www.wikidata.org/entity/Q227'), ('country', 'Azerbaijan'), ('noOfPhysicists', '1')]
[('countryIRI', 'http://www.wikidata.org/entity/Q184'), ('country', 'Belarus'), ('noOfPhysicists', '1')]
[('countryIRI', 'http://www.wikidata.org/entity/Q31'), ('country', 'Belgium'), ('noOfPhysicists', '1')]
[('countryIRI', 'http://www.wikidata.org/entity/Q213'), ('country', 'Czech Republic'), ('noOfPhysicists', '1')]
[('countryIRI', 'http://www.wikidata.org/entity/Q756617'), ('country', 'Danish Realm'), ('noOfPhysicists', '2')]
[('countryIRI', 'http://www.wikidata.org/entity/Q142'), ('country', 'France'), ('noOfPhysicists', '9')]
[('countryIRI', 'http://www.wikidata.org/entity/Q183'), ('country', 'Germany'), ('noOfPhysicists', '26')]
[('countryIRI', 'http://www.wikidata.org/entity/Q28'), ('country', 'Hungary'), ('noOfPhysicists', '2')]
[('countryIRI', 'http://www.wiki

In [61]:
# Query 18

# Query 17 is the result of Task 4, because it said: "Consider physicists born in Europe...".
# If we should return 0 for countries they never won the 'Nobel Prize in Physics', this query is the result.

# It is supposed that 'physicist' means that the 'occupation' of the winner should be 'physicist'. I do not check if there is any person who won the Nobel Prize in Physics whos occupation is not physicist.

# wdt:P166 => 'award received'
# wd:Q38104 => 'Nobel Prize in Physics'
# wd:P106 => 'occupation'
# wd:Q169470 => 'physicist'
# wdt:P19 => 'place of birth'
# wdt:P17 => 'country'
# wdt:P30 => 'continent'
# wd:Q46 => 'Europe'


queryString = """
SELECT DISTINCT (?c as ?countryIRI) (?cName as ?country) (count(?s) as ?noOfPhysicists)
WHERE {
   # bind something
   ?c wdt:P30 wd:Q46 ;
      wdt:P31 wd:Q6256 .
   OPTIONAL
   {
       ?s wdt:P166 wd:Q38104 ;
          wdt:P106 wd:Q169470 ;
          wdt:P19 ?pBirth .
      ?pBirth wdt:P17 ?c .
  }

   # get the label
   ?c sc:name ?cName .

}
GROUP BY (?c) (?cName)
ORDER BY (?cName)
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('countryIRI', 'http://www.wikidata.org/entity/Q222'), ('country', 'Albania'), ('noOfPhysicists', '0')]
[('countryIRI', 'http://www.wikidata.org/entity/Q228'), ('country', 'Andorra'), ('noOfPhysicists', '0')]
[('countryIRI', 'http://www.wikidata.org/entity/Q40'), ('country', 'Austria'), ('noOfPhysicists', '2')]
[('countryIRI', 'http://www.wikidata.org/entity/Q184'), ('country', 'Belarus'), ('noOfPhysicists', '1')]
[('countryIRI', 'http://www.wikidata.org/entity/Q219'), ('country', 'Bulgaria'), ('noOfPhysicists', '0')]
[('countryIRI', 'http://www.wikidata.org/entity/Q224'), ('country', 'Croatia'), ('noOfPhysicists', '0')]
[('countryIRI', 'http://www.wikidata.org/entity/Q229'), ('country', 'Cyprus'), ('noOfPhysicists', '0')]
[('countryIRI', 'http://www.wikidata.org/entity/Q213'), ('country', 'Czech Republic'), ('noOfPhysicists', '1')]
[('countryIRI', 'http://www.wikidata.org/entity/Q756617'), ('country', 'Danish Realm'), ('noOfPhysicists', '2')]
[('countryIRI', 'http://www.wikid

Final query for this task

In [58]:
# write your final query

# Query 19

# Task 4:
# Consider physicists born in Europe.
# For each european country return the number of physicists who won the Nobel Prize in Physics (the result set must be a list of triples country IRI, label and #physicists).

# It is supposed that 'physicist' means that the 'occupation' of the winner should be 'physicist'. I do not check if there is any person who won the Nobel Prize in Physics whos occupation is not physicist.

# wdt:P166 => 'award received'
# wd:Q38104 => 'Nobel Prize in Physics'
# wd:P106 => 'occupation'
# wd:Q169470 => 'physicist'
# wdt:P19 => 'place of birth'
# wdt:P17 => 'country'
# wdt:P30 => 'continent'
# wd:Q46 => 'Europe'


queryString = """
SELECT DISTINCT (?c as ?countryIRI) (?cName as ?country) (count(?s) as ?noOfPhysicists)
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
      wdt:P106 wd:Q169470 ;
      wdt:P19 ?pBirth .
  ?pBirth wdt:P17 ?c .
  ?c wdt:P30 wd:Q46 .

   # get the label
   ?c sc:name ?cName .

}
GROUP BY (?c) (?cName)
ORDER BY (?cName)
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('countryIRI', 'http://www.wikidata.org/entity/Q40'), ('country', 'Austria'), ('noOfPhysicists', '2')]
[('countryIRI', 'http://www.wikidata.org/entity/Q227'), ('country', 'Azerbaijan'), ('noOfPhysicists', '1')]
[('countryIRI', 'http://www.wikidata.org/entity/Q184'), ('country', 'Belarus'), ('noOfPhysicists', '1')]
[('countryIRI', 'http://www.wikidata.org/entity/Q31'), ('country', 'Belgium'), ('noOfPhysicists', '1')]
[('countryIRI', 'http://www.wikidata.org/entity/Q213'), ('country', 'Czech Republic'), ('noOfPhysicists', '1')]
[('countryIRI', 'http://www.wikidata.org/entity/Q756617'), ('country', 'Danish Realm'), ('noOfPhysicists', '2')]
[('countryIRI', 'http://www.wikidata.org/entity/Q142'), ('country', 'France'), ('noOfPhysicists', '9')]
[('countryIRI', 'http://www.wikidata.org/entity/Q183'), ('country', 'Germany'), ('noOfPhysicists', '26')]
[('countryIRI', 'http://www.wikidata.org/entity/Q28'), ('country', 'Hungary'), ('noOfPhysicists', '2')]
[('countryIRI', 'http://www.wiki

## Task 5
Return the countries which won at least once all the type of Nobel Prizes with the people born there. (the result set must be a list of couples country IRI and label).

In [64]:
# write your queries

# Query 20

# Finding all types of Nobel Prize
# wd:Q7191 => "Nobel Prize"

queryString = """
SELECT DISTINCT ?s ?sName
WHERE {
   # bind something
   ?s wdt:P279 wd:Q7191 .
   
   # get the label
   ?s sc:name ?sName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q35637'), ('sName', 'Nobel Peace Prize')]
[('s', 'http://www.wikidata.org/entity/Q37922'), ('sName', 'Nobel Prize in Literature')]
[('s', 'http://www.wikidata.org/entity/Q38104'), ('sName', 'Nobel Prize in Physics')]
[('s', 'http://www.wikidata.org/entity/Q44585'), ('sName', 'Nobel Prize in Chemistry')]
[('s', 'http://www.wikidata.org/entity/Q47170'), ('sName', 'Prize in Economic Sciences in Memory of Alfred Nobel')]
[('s', 'http://www.wikidata.org/entity/Q80061'), ('sName', 'Nobel Prize in Physiology or Medicine')]
6


In [81]:
# Query 21

# Finding total number of distinct Nobel Prize each European country won.
# wdt:P166 => 'award received'
# wdt:P19 => 'place of birth'
# wdt:P17 => 'country'


queryString = """
SELECT DISTINCT (?c as ?countryIRI) (?cName as ?country) (COUNT(DISTINCT(?prize)) as ?noOfPrizes) 
WHERE {
   # bind something
   ?s wdt:P166 ?prize ;
      wdt:P19 ?pBirth .
      
   ?prize wdt:P279 wd:Q7191 .
   
   ?pBirth wdt:P17 ?c .

   # get the label
   ?c sc:name ?cName .

}
GROUP BY (?c) (?cName)
ORDER BY (?cName)
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('countryIRI', 'http://www.wikidata.org/entity/Q262'), ('country', 'Algeria'), ('noOfPrizes', '2')]
[('countryIRI', 'http://www.wikidata.org/entity/Q414'), ('country', 'Argentina'), ('noOfPrizes', '2')]
[('countryIRI', 'http://www.wikidata.org/entity/Q408'), ('country', 'Australia'), ('noOfPrizes', '3')]
[('countryIRI', 'http://www.wikidata.org/entity/Q40'), ('country', 'Austria'), ('noOfPrizes', '6')]
[('countryIRI', 'http://www.wikidata.org/entity/Q28513'), ('country', 'Austria-Hungary'), ('noOfPrizes', '1')]
[('countryIRI', 'http://www.wikidata.org/entity/Q131964'), ('country', 'Austrian Empire'), ('noOfPrizes', '1')]
[('countryIRI', 'http://www.wikidata.org/entity/Q227'), ('country', 'Azerbaijan'), ('noOfPrizes', '1')]
[('countryIRI', 'http://www.wikidata.org/entity/Q902'), ('country', 'Bangladesh'), ('noOfPrizes', '1')]
[('countryIRI', 'http://www.wikidata.org/entity/Q184'), ('country', 'Belarus'), ('noOfPrizes', '3')]
[('countryIRI', 'http://www.wikidata.org/entity/Q31')

In [60]:
# Query 22

# From Query 20 I know that total number of distinct Nobel Prizes is 6.
# From noOfPrizes of Query 21 it is possible to find the result. Just need to check which countries won 6 distinct Nobel Prizes. 
# Some countries appear in the result in different rows, because of their different name during history, or they won a prize with other countries.
# Austria & Germany --> appear in the result in any case because they have noOfPrizes = 6 for at least one row.
# China appears in two rows with noOfprizes = 1 & 4. So in the best scenario final noOfPrizes could reach 5. No appearence in the final result.
# Hungry & Netherlands should be check.

# wdt:P166 => 'award received'
# wdt:P19 => 'place of birth'
# wdt:P17 => 'country'

queryString = """
SELECT DISTINCT (?cName as ?country)  ?prizeName
WHERE {
   # bind something
   ?s wdt:P166 ?prize ;
      wdt:P19 ?pBirth .
      
   ?prize wdt:P279 wd:Q7191 .
   
   ?pBirth wdt:P17 ?c .

   # get the label
   ?c sc:name ?cName .
   ?prize sc:name ?prizeName
   
   FILTER (REGEX(?cName,"Hungary") || REGEX(?cName,"Netherlands"))

}
GROUP BY (?c) (?cName)
ORDER BY (?cName)
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'Austria-Hungary'), ('prizeName', 'Nobel Prize in Chemistry')]
[('country', 'Hungary'), ('prizeName', 'Nobel Prize in Physiology or Medicine')]
[('country', 'Hungary'), ('prizeName', 'Nobel Prize in Physics')]
[('country', 'Hungary'), ('prizeName', 'Nobel Prize in Chemistry')]
[('country', 'Hungary'), ('prizeName', 'Prize in Economic Sciences in Memory of Alfred Nobel')]
[('country', 'Hungary'), ('prizeName', 'Nobel Prize in Literature')]
[('country', 'Kingdom of the Netherlands'), ('prizeName', 'Nobel Prize in Physics')]
[('country', 'Kingdom of the Netherlands'), ('prizeName', 'Nobel Prize in Chemistry')]
[('country', 'Kingdom of the Netherlands'), ('prizeName', 'Nobel Peace Prize')]
[('country', 'Netherlands'), ('prizeName', 'Prize in Economic Sciences in Memory of Alfred Nobel')]
[('country', 'Netherlands'), ('prizeName', 'Nobel Peace Prize')]
[('country', 'Netherlands'), ('prizeName', 'Nobel Prize in Physiology or Medicine')]
[('country', 'Netherlands'), ('pri

Final query for this task

In [92]:
# write your final query

# Query 23

# From Query 20 I know that total number of distinct Nobel Prizes is 6.
# From noOfPrizes of Query 21 it is possible to find the result.
# The result of Query 22 shows that issues about Hungary and Netherlands do not change the final result (They should not appear in the final result).

# wdt:P166 => 'award received'
# wdt:P19 => 'place of birth'
# wdt:P17 => 'country'

queryString = """
SELECT DISTINCT (?c as ?countryIRI) (?cName as ?country)  
WHERE {
   # bind something
   ?s wdt:P166 ?prize ;
      wdt:P19 ?pBirth .
      
   ?prize wdt:P279 wd:Q7191 .
   
   ?pBirth wdt:P17 ?c .

   # get the label
   ?c sc:name ?cName .

}
GROUP BY (?c) (?cName)
HAVING (COUNT(DISTINCT(?prize)) = 6)
ORDER BY (?cName)
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('countryIRI', 'http://www.wikidata.org/entity/Q40'), ('country', 'Austria')]
[('countryIRI', 'http://www.wikidata.org/entity/Q16'), ('country', 'Canada')]
[('countryIRI', 'http://www.wikidata.org/entity/Q142'), ('country', 'France')]
[('countryIRI', 'http://www.wikidata.org/entity/Q183'), ('country', 'Germany')]
[('countryIRI', 'http://www.wikidata.org/entity/Q668'), ('country', 'India')]
[('countryIRI', 'http://www.wikidata.org/entity/Q38'), ('country', 'Italy')]
[('countryIRI', 'http://www.wikidata.org/entity/Q20'), ('country', 'Norway')]
[('countryIRI', 'http://www.wikidata.org/entity/Q36'), ('country', 'Poland')]
[('countryIRI', 'http://www.wikidata.org/entity/Q159'), ('country', 'Russia')]
[('countryIRI', 'http://www.wikidata.org/entity/Q34'), ('country', 'Sweden')]
[('countryIRI', 'http://www.wikidata.org/entity/Q145'), ('country', 'United Kingdom')]
[('countryIRI', 'http://www.wikidata.org/entity/Q30'), ('country', 'United States of America')]
12


## Task 6
Amongst the physicists who won the Nobel Prize in Physics, find who had discovered or invented at least 3 things (the result set must be a list of triples physicist IRI, label and #discoveries ).

In [112]:
# write your queries

# Query 24

# All information is founded before.

# P61 => "discoverer or inventor"
# wd:Q169470 => 'physicist'
# wd:Q38104 => 'Nobel Prize in Physics'
# wdt:P166 => 'award received'  (from Query 7)
# wd:Q937 => 'Albert Einstein'
# wdt:P106 => 'occupation'

queryString = """
SELECT DISTINCT ?s  ?sName COUNT(?o) AS ?noOfDiscoveries
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
      wdt:P106 wd:Q169470 .
   
   ?o wdt:P61 ?s .
      
   # get the label
   ?s sc:name ?sName .

}
GROUP BY (?s) (?sName)
HAVING (COUNT(?o) > 2)
ORDER BY (?sName)
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q937'), ('sName', 'Albert Einstein'), ('noOfDiscoveries', '10')]
[('s', 'http://www.wikidata.org/entity/Q173028'), ('sName', 'Emilio G. Segrè'), ('noOfDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q8753'), ('sName', 'Enrico Fermi'), ('noOfDiscoveries', '6')]
[('s', 'http://www.wikidata.org/entity/Q160518'), ('sName', 'Gustaf Dalén'), ('noOfDiscoveries', '4')]
[('s', 'http://www.wikidata.org/entity/Q47285'), ('sName', 'J. J. Thomson'), ('noOfDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q123975'), ('sName', 'Michel Mayor'), ('noOfDiscoveries', '12')]
[('s', 'http://www.wikidata.org/entity/Q47480'), ('sName', 'Paul Dirac'), ('noOfDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q37463'), ('sName', 'Pierre Curie'), ('noOfDiscoveries', '4')]
[('s', 'http://www.wikidata.org/entity/Q193803'), ('sName', 'Roger Penrose'), ('noOfDiscoveries', '6')]
9


Final query for this task

In [114]:
# write your final query


# Query 25

# All information is founded before.

# P61 => "discoverer or inventor"
# wd:Q169470 => 'physicist'
# wd:Q38104 => 'Nobel Prize in Physics'
# wdt:P166 => 'award received'  (from Query 7)
# wd:Q937 => 'Albert Einstein'
# wdt:P106 => 'occupation'

queryString = """
SELECT DISTINCT ?s AS ?physiscistIRI  ?sName as ?physiscistName COUNT(?o) AS ?noOfDiscoveries
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104 ;
      wdt:P106 wd:Q169470 .
   
   ?o wdt:P61 ?s .
      
   # get the label
   ?s sc:name ?sName .

}
GROUP BY (?s) (?sName)
HAVING (COUNT(?o) > 2)
ORDER BY (?sName)
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('physiscistIRI', 'http://www.wikidata.org/entity/Q937'), ('physiscistName', 'Albert Einstein'), ('noOfDiscoveries', '10')]
[('physiscistIRI', 'http://www.wikidata.org/entity/Q173028'), ('physiscistName', 'Emilio G. Segrè'), ('noOfDiscoveries', '3')]
[('physiscistIRI', 'http://www.wikidata.org/entity/Q8753'), ('physiscistName', 'Enrico Fermi'), ('noOfDiscoveries', '6')]
[('physiscistIRI', 'http://www.wikidata.org/entity/Q160518'), ('physiscistName', 'Gustaf Dalén'), ('noOfDiscoveries', '4')]
[('physiscistIRI', 'http://www.wikidata.org/entity/Q47285'), ('physiscistName', 'J. J. Thomson'), ('noOfDiscoveries', '3')]
[('physiscistIRI', 'http://www.wikidata.org/entity/Q123975'), ('physiscistName', 'Michel Mayor'), ('noOfDiscoveries', '12')]
[('physiscistIRI', 'http://www.wikidata.org/entity/Q47480'), ('physiscistName', 'Paul Dirac'), ('noOfDiscoveries', '3')]
[('physiscistIRI', 'http://www.wikidata.org/entity/Q37463'), ('physiscistName', 'Pierre Curie'), ('noOfDiscoveries', '4')]
[

## Task 7
Amongst the physicists who won the Nobel Prize in Physics, find who taught in the university of his/her city of birth (the result set must be a list of elements of the form physicist IRI, label, university IRI and label).

In [131]:
# write your queries

# Query 25

# Looking for relation between person and university. I cheked any things having 'university' related to 'Albert Einstein'.

# What I know till now:
# P61 => "discoverer or inventor"
# wd:Q169470 => 'physicist'
# wd:Q38104 => 'Nobel Prize in Physics'
# wdt:P166 => 'award received'  (from Query 7)
# wdt:P106 => 'occupation'
# wd:Q1622272 => 'university teacher' (from Query 5)
# wd:Q37226 => 'teacher' (from Query 5)
# wd:Q937 => 'Albert Einstein'


queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q937 ?p ?o .
   
   # get the label
   ?p sc:name ?pName .
   ?o sc:name ?oName .
   
   FILTER (REGEX (?pName,"[Uu]niversity") || REGEX (?oName,"[Uu]niversity"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q659080'), ('oName', 'University of Bern')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q1622272'), ('oName', 'university teacher')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q168756'), ('oName', 'University of California, Berkeley')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('o', 'http://www.wikidata.org/entity/Q25287226'), ('oName', 'Honorary doctor of the University of Geneva')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('o', 'http://www.wikidata.org/entity/Q28861731'), ('oName', 'honorary doctor of the Hebrew University of Jerusalem')]
[('p', 'http://www.wikidata.org/prop/direct/P485'), ('pName', 'archives at'), ('o', 'http://www.wikidata.org/ent

In [33]:

# Query 25-improved

# In Query 25 I searched (FILTER) both prediction and object for '[Uu]niversity'.
# But since it is supposed that the triple be somthing like "Einstein  taught in university XX", searching prediction for '[Uu]niversity' is an unnecessary computation load.
# Thus it is better to search just object. I improved the Query 25 as this one.


# Looking for relation between person and university. I cheked any things having 'university' related to 'Albert Einstein'.

# What I know till now:
# P61 => "discoverer or inventor"
# wd:Q169470 => 'physicist'
# wd:Q38104 => 'Nobel Prize in Physics'
# wdt:P166 => 'award received'  (from Query 7)
# wdt:P106 => 'occupation'
# wd:Q1622272 => 'university teacher' (from Query 5)
# wd:Q37226 => 'teacher' (from Query 5)
# wd:Q937 => 'Albert Einstein'


queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q937 ?p ?o .
   
   # get the label
   ?p sc:name ?pName .
   ?o sc:name ?oName .
   
   FILTER (REGEX (?oName,"[Uu]niversity"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q1622272'), ('oName', 'university teacher')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q1729754'), ('oName', 'German University in Prague')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q156598'), ('oName', 'Leiden University')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q21578'), ('oName', 'Princeton University')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q659080'), ('oName', 'University of Bern')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q206702'), ('oName', 'University of Zurich')]
[('p', 'http://www.wikidata.org/pro

In [61]:
# Query 26

# Lets see if wdt:P108 ('employer') is a good choice.

# What I know till now:
# P61 => "discoverer or inventor"
# wd:Q169470 => 'physicist'
# wd:Q38104 => 'Nobel Prize in Physics'
# wdt:P166 => 'award received'  (from Query 7)
# wdt:P106 => 'occupation'
# wd:Q1622272 => 'university teacher' (from Query 5)
# wd:Q37226 => 'teacher' (from Query 5)
# wd:Q937 => 'Albert Einstein'
# wdt:P108 => 'employer'
# wdt:P19 => 'place of birth'


queryString = """
SELECT DISTINCT  ?sName ?o ?oName ?pBirth ?birthCity
WHERE {
   # bind something
  
   ?s wdt:P166 wd:Q38104 ;
      wdt:P106 wd:Q169470 ;
      wdt:P19 ?pBirth ;
      wdt:P108 ?o .
   
   # get the label
   ?s sc:name ?sName .
   ?o sc:name ?oName .
   ?pBirth sc:name ?birthCity .
}

LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('sName', 'William Henry Bragg'), ('o', 'http://www.wikidata.org/entity/Q15574'), ('oName', 'University of Adelaide'), ('pBirth', 'http://www.wikidata.org/entity/Q2227457'), ('birthCity', 'Wigton')]
[('sName', 'Donna Strickland'), ('o', 'http://www.wikidata.org/entity/Q1437507'), ('oName', 'National Research Council'), ('pBirth', 'http://www.wikidata.org/entity/Q504114'), ('birthCity', 'Guelph')]
[('sName', 'Brian David Josephson'), ('o', 'http://www.wikidata.org/entity/Q4380853'), ('oName', 'Missouri University of Science and Technology'), ('pBirth', 'http://www.wikidata.org/entity/Q10690'), ('birthCity', 'Cardiff')]
[('sName', 'Alexei Abrikosov'), ('o', 'http://www.wikidata.org/entity/Q492766'), ('oName', 'N. I. Lobachevsky State University of Nizhny Novgorod'), ('pBirth', 'http://www.wikidata.org/entity/Q649'), ('birthCity', 'Moscow')]
[('sName', 'Henri Becquerel'), ('o', 'http://www.wikidata.org/entity/Q838691'), ('oName', "Muséum national d'histoire naturelle"), ('pBirth'

In [63]:
# Query 27

# Looking for city of university. To do this, I check all properties of a sample university (the first one from the result of Query 26).

# wd:Q515 => 'city' (from Query 16)
# wd:Q15574 => 'University of Adelaide'
# wdt:P19 => 'place of birth'

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
  
   wd:Q15574 ?p ?o .
   # get the label
   ?p sc:name ?pName .

}
ORDER BY (?pName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5242'), ('pName', 'ARWU university ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6683'), ('pName', 'Alexander Turnbull Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('pName', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2651'), ('pName', 'CRICOS Provider Code')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P3153'), ('pName', 'Crossref funder ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5785'), ('pName', 'EU Research participant ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3608'), ('pName', 'EU VAT number')]
[('p', 'http://www.wikidata.org/prop/direct/P4228'), ('pName', 'Encyclopedia of Australian Science ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('pName', 'Faceb

In [65]:
# Query 28

# Some predicates contain 'location'. Let filter them.

# wdt:P => 'headquarters location'
# wd:Q515 => 'city' (from Query 16)
# wd:Q15574 => 'University of Adelaide'
# wd:Q161976 => 'University of Würzburg'
# wdt:P19 => 'place of birth'

queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
  
   wd:Q15574 ?p ?o .
   
   # get the label
   ?p sc:name ?pName .
   
   
   FILTER(REGEX(?pName,"[Ll]ocation"))  

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location')]
2


In [35]:
# Query 29

# Many of universities have branches, departments or buildings in different cities.
# Considering this fact, it seems that choosing wd:P159 ('headquarters location') is a good idea.

# wdt:P159 => 'headquarters location'
# wd:Q515 => 'city'
# wd:Q15574 => 'University of Adelaide'
# wdt:P19 => 'place of birth'
# P61 => "discoverer or inventor"
# wd:Q169470 => 'physicist'
# wd:Q38104 => 'Nobel Prize in Physics'
# wdt:P166 => 'award received'  (from Query 7)
# wdt:P106 => 'occupation'
# wd:Q1622272 => 'university teacher' (from Query 5)
# wd:Q37226 => 'teacher' (from Query 5)
# wd:Q937 => 'Albert Einstein'
# wdt:P108 => 'employer'
# wdt:P19 => 'place of birth'

queryString = """
SELECT DISTINCT ?sName ?oName ?cName AS ?universityCity ?birthCity
WHERE {
   # bind something
  
   ?s wdt:P166 wd:Q38104 ;
      wdt:P106 wd:Q169470 ;
      wdt:P19 ?pBirth ;
      wdt:P108 ?o .
      
   ?o wdt:P159 ?c .
   
   # get the label
   ?s sc:name ?sName .
   ?o sc:name ?oName .
   ?c sc:name ?cName .
   ?pBirth sc:name ?birthCity .
}
ORDER BY (?sName)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('sName', 'Aage Niels Bohr'), ('oName', 'Los Alamos National Laboratory'), ('universityCity', 'Los Alamos'), ('birthCity', 'Copenhagen')]
[('sName', 'Aage Niels Bohr'), ('oName', 'Nordic Institute for Theoretical Physics'), ('universityCity', 'Stockholm'), ('birthCity', 'Copenhagen')]
[('sName', 'Adam Riess'), ('oName', 'University of California, Berkeley'), ('universityCity', 'Berkeley'), ('birthCity', 'Washington, D.C.')]
[('sName', 'Adam Riess'), ('oName', 'Johns Hopkins University'), ('universityCity', 'Baltimore'), ('birthCity', 'Washington, D.C.')]
[('sName', 'Adam Riess'), ('oName', 'Space Telescope Science Institute'), ('universityCity', 'Baltimore'), ('birthCity', 'Washington, D.C.')]
[('sName', 'Albert A. Michelson'), ('oName', 'Case Western Reserve University'), ('universityCity', 'Cleveland'), ('birthCity', 'Strzelno')]
[('sName', 'Albert Einstein'), ('oName', 'Deutsche Physikalische Gesellschaft'), ('universityCity', 'Bad Honnef'), ('birthCity', 'Ulm')]
[('sName',

In [51]:
# Query 30

# It seems that everything is ready to run final query.

# Task 7:
# Amongst the physicists who won the Nobel Prize in Physics,
# find who taught in the university of his/her city of birth (the result set must be a list of elements of the form physicist IRI, label, university IRI and label).


# wdt:P159 => 'headquarters location'
# wd:Q515 => 'city'
# wd:Q15574 => 'University of Adelaide'
# wdt:P19 => 'place of birth'
# P61 => "discoverer or inventor"
# wd:Q169470 => 'physicist'
# wd:Q38104 => 'Nobel Prize in Physics'
# wdt:P166 => 'award received'  (from Query 7)
# wdt:P106 => 'occupation'
# wd:Q1622272 => 'university teacher' (from Query 5)
# wd:Q37226 => 'teacher' (from Query 5)
# wd:Q937 => 'Albert Einstein'
# wdt:P108 => 'employer'
# wdt:P19 => 'place of birth'

queryString = """
SELECT DISTINCT ?s ?sName ?university ?universityName
WHERE {
   # bind something
  
   ?s wdt:P166 wd:Q38104 ;
      wdt:P106 wd:Q169470 ;
      wdt:P19 ?pBirth ;
      wdt:P108 ?university .
      
   ?university wdt:P159 ?pBirth.
   
   # get the label
   ?s sc:name ?sName .
   ?university sc:name ?universityName .

}
ORDER BY (?sName) (?universityName)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q188128'), ('sName', 'Alexei Abrikosov'), ('university', 'http://www.wikidata.org/entity/Q1472245'), ('universityName', 'Bauman Moscow State Technical University')]
[('s', 'http://www.wikidata.org/entity/Q188128'), ('sName', 'Alexei Abrikosov'), ('university', 'http://www.wikidata.org/entity/Q13164'), ('universityName', 'Moscow State University')]
[('s', 'http://www.wikidata.org/entity/Q188128'), ('sName', 'Alexei Abrikosov'), ('university', 'http://www.wikidata.org/entity/Q4315005'), ('universityName', 'National University of Science and Technology')]
[('s', 'http://www.wikidata.org/entity/Q9130'), ('sName', 'Erwin Schrödinger'), ('university', 'http://www.wikidata.org/entity/Q165980'), ('universityName', 'University of Vienna')]
[('s', 'http://www.wikidata.org/entity/Q41269'), ('sName', 'Henri Becquerel'), ('university', 'http://www.wikidata.org/entity/Q838691'), ('universityName', "Muséum national d'histoire naturelle")]
[('s', 'http://

Final query for this task

In [55]:
# write your final query

# Query 31

# wdt:P166 => 'award received'
# wd:Q38104 => 'Nobel Prize in Physics'
# wdt:P106 => 'occupation'
# wd:Q169470 => 'physicist'
# wdt:P19 => 'place of birth'
# wdt:P108 => 'employer'
# wdt:P159 => 'headquarters location'


queryString = """
SELECT DISTINCT ?s AS ?IRI ?Name ?uni ?uniName
WHERE {
   # bind something
  
   ?s wdt:P166 wd:Q38104 ;
      wdt:P106 wd:Q169470 ;
      wdt:P19 ?pBirth ;
      wdt:P108 ?uni .
      
   ?uni wdt:P159 ?pBirth.
   
   # get the label
   ?s sc:name ?Name .
   ?uni sc:name ?uniName .

}
ORDER BY (?physicistName) (?universityName)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q202164'), ('Name', 'Makoto Kobayashi'), ('uni', 'http://www.wikidata.org/entity/Q1191132'), ('uniName', 'Nagoya University')]
[('IRI', 'http://www.wikidata.org/entity/Q9130'), ('Name', 'Erwin Schrödinger'), ('uni', 'http://www.wikidata.org/entity/Q165980'), ('uniName', 'University of Vienna')]
[('IRI', 'http://www.wikidata.org/entity/Q184499'), ('Name', 'Patrick Blackett, Baron Blackett'), ('uni', 'http://www.wikidata.org/entity/Q375606'), ('uniName', 'Birkbeck, University of London')]
[('IRI', 'http://www.wikidata.org/entity/Q41269'), ('Name', 'Henri Becquerel'), ('uni', 'http://www.wikidata.org/entity/Q838691'), ('uniName', "Muséum national d'histoire naturelle")]
[('IRI', 'http://www.wikidata.org/entity/Q9021'), ('Name', 'Max Planck'), ('uni', 'http://www.wikidata.org/entity/Q156737'), ('uniName', 'University of Kiel')]
[('IRI', 'http://www.wikidata.org/entity/Q172118'), ('Name', 'Percy Williams Bridgman'), ('uni', 'http://www.wikida